In [ ]:
# | default_exp _application.tester

In [ ]:
# | export

import asyncio
import collections
import inspect
from contextlib import asynccontextmanager
from itertools import groupby
from typing import *

from pydantic import BaseModel

from fastkafka import KafkaEvent
from fastkafka._application.app import FastKafka, AwaitedMock
from fastkafka._components.asyncapi import KafkaBroker
from fastkafka._components.helpers import unwrap_list_type
from fastkafka._components.meta import delegates, export, patch
from fastkafka._components.producer_decorator import unwrap_from_kafka_event
from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker
from fastkafka._testing.in_memory_broker import InMemoryBroker
from fastkafka._testing.local_redpanda_broker import LocalRedpandaBroker

In [ ]:
import pytest
from pydantic import Field

from fastkafka import EventMetadata
from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | notest
# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [ ]:
# | export


def _get_broker_spec(bootstrap_server: str) -> KafkaBroker:
    url = bootstrap_server.split(":")[0]
    port = bootstrap_server.split(":")[1]
    return KafkaBroker(url=url, port=port)

## Fastkafka Tester class

In [ ]:
# | export


@export("fastkafka.testing")
class Tester(FastKafka):
    __test__ = False

    @delegates(ApacheKafkaBroker.__init__)
    def __init__(
        self,
        app: Union[FastKafka, List[FastKafka]],
        *,
        broker: Optional[
            Union[ApacheKafkaBroker, LocalRedpandaBroker, InMemoryBroker]
        ] = None,
    ):
        """Mirror-like object for testing a FastFafka application

        Can be used as context manager

        """
        self.apps = app if isinstance(app, list) else [app]

        super().__init__()
        self.mirrors: Dict[Any, Any] = {}
        self.create_mirrors()

        self.broker = broker
        
        unique_broker_configs = []
        for app in self.apps:
            for broker_config in app._override_brokers:
                if broker_config not in unique_broker_configs:
                    unique_broker_configs.append(broker_config)
        self.num_brokers = len(unique_broker_configs)
        
        self.overriden_brokers: List[Union[ApacheKafkaBroker, LocalRedpandaBroker]] = []

    @delegates(LocalRedpandaBroker.__init__)
    def using_local_redpanda(self, **kwargs: Any) -> "Tester":
        """Starts local Redpanda broker used by the Tester instance

        Args:
            listener_port: Port on which the clients (producers and consumers) can connect
            tag: Tag of Redpanda image to use to start container
            seastar_core: Core(s) to use byt Seastar (the framework Redpanda uses under the hood)
            memory: The amount of memory to make available to Redpanda
            mode: Mode to use to load configuration properties in container
            default_log_level: Log levels to use for Redpanda
            topics: List of topics to create after sucessfull redpanda broker startup
            retries: Number of retries to create redpanda service
            apply_nest_asyncio: set to True if running in notebook
            port allocation if the requested port was taken

        Returns:
            An instance of tester with Redpanda as broker
        """
        topics = set().union(*(app.get_topics() for app in self.apps))
        kwargs["topics"] = (
            topics.union(kwargs["topics"]) if "topics" in kwargs else topics
        )
        self.broker = LocalRedpandaBroker(**kwargs)
        self.overriden_brokers = [
            LocalRedpandaBroker(**kwargs) for _ in range(self.num_brokers)
        ]
        return self

    @delegates(ApacheKafkaBroker.__init__)
    def using_local_kafka(self, **kwargs: Any) -> "Tester":
        """Starts local Kafka broker used by the Tester instance

        Args:
            data_dir: Path to the directory where the zookeepeer instance will save data
            zookeeper_port: Port for clients (Kafka brokes) to connect
            listener_port: Port on which the clients (producers and consumers) can connect
            topics: List of topics to create after sucessfull Kafka broker startup
            retries: Number of retries to create kafka and zookeeper services using random
            apply_nest_asyncio: set to True if running in notebook
            port allocation if the requested port was taken

        Returns:
            An instance of tester with Kafka as broker
        """
        topics = set().union(*(app.get_topics() for app in self.apps))
        kwargs["topics"] = (
            topics.union(kwargs["topics"]) if "topics" in kwargs else topics
        )
        self.broker = ApacheKafkaBroker(**kwargs)
        self.overriden_brokers = [
            ApacheKafkaBroker(**kwargs) for _ in range(self.num_brokers)
        ]

        return self

    async def _start_tester(self) -> None:
        """Starts the Tester"""
        for app in self.apps:
            app.create_mocks()
            await app.__aenter__()
        self.create_mocks()
        await super().__aenter__()
        await asyncio.sleep(3)

    async def _stop_tester(self) -> None:
        """Shuts down the Tester"""
        await super().__aexit__(None, None, None)
        for app in self.apps[::-1]:
            await app.__aexit__(None, None, None)

    def create_mirrors(self) -> None:
        pass

    @asynccontextmanager
    async def _create_ctx(self) -> AsyncGenerator["Tester", None]:
        if self.broker is None:
            topics = set().union(*(app.get_topics() for app in self.apps))
            self.broker = InMemoryBroker()

        broker_spec = _get_broker_spec(await self.broker._start())

        try:
            if isinstance(self.broker, (ApacheKafkaBroker, LocalRedpandaBroker)):
                override_broker_configs = [
                    list(grp)
                    for k, grp in groupby(
                        [
                            broker_config
                            for app in self.apps + [self]
                            for broker_config in app._override_brokers
                        ]
                    )
                ]

                for override_brokers_config_groups, broker in zip(
                    override_broker_configs, self.overriden_brokers
                ):
                    b_s = _get_broker_spec(await broker._start())
                    for override_broker_config in override_brokers_config_groups:
                        override_broker_config["fastkafka_tester_broker"] = b_s

                for app in self.apps + [self]:
                    app._kafka_brokers.brokers["fastkafka_tester_broker"] = broker_spec
                    app.set_kafka_broker("fastkafka_tester_broker")
            await self._start_tester()
            try:
                yield self
            finally:
                await self._stop_tester()
        finally:
            await self.broker._stop()
            for broker in self.overriden_brokers:
                await broker._stop()

    async def __aenter__(self) -> "Tester":
        self._ctx = self._create_ctx()
        return await self._ctx.__aenter__()

    async def __aexit__(self, *args: Any) -> None:
        await self._ctx.__aexit__(*args)

In [ ]:
for _ in range(2):
    with pytest.raises(RuntimeError) as e:
        async with Tester(app) as tester:
            assert tester.broker.is_started
            assert tester.is_started
            raise RuntimeError("ok")

    print(e)
    assert not tester.broker.is_started
    assert not tester.is_started

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[]
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] fa

In [ ]:
tester_with_redpanda = Tester(app).using_local_redpanda(tag="v22.3.15")
assert isinstance(tester_with_redpanda.broker, LocalRedpandaBroker)

In [ ]:
tester = Tester(app)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[]
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_s

In [ ]:
tester = Tester(app).using_local_kafka(zookeeper_port=9998, listener_port=9788)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

{'preprocessed_signals', 'predictions'}
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9788
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[]
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9788'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop

## Test overriding brokers

In [ ]:
kafka_brokers = dict(localhost=[dict(url="some_server", port=9092)])


overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", brokers=kafka_brokers)
async def on_preprocessed_signals_overriden(msg: TestMsg):
    print(f"Overriden:  {msg=}")


@overriden_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    print(f"Original:  {msg=}")


tester = Tester(overriden_app)


@tester.produces(brokers=kafka_brokers)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals_overriden.assert_called(
        timeout=5
    )
    await overriden_app.awaited_mocks.on_preprocessed_signals.assert_not_called(
        timeout=5
    )

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[{'localhost': [KafkaBroker(url='some_server', description='Kafka broker', port='9092', protocol='kafka', security=None)]}]
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['some_server:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': ['some_server:9092']}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer pa

In [ ]:
kafka_brokers = dict(localhost=[dict(url="some_server", port=9092)])

overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", brokers=kafka_brokers)
async def on_preprocessed_signals_overriden(msg: TestMsg):
    print(f"Overriden:  {msg=}")


@overriden_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    print(f"Original:  {msg=}")


tester = Tester(overriden_app).using_local_kafka()


@tester.produces(brokers=kafka_brokers)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg

{'preprocessed_signals'}


In [ ]:
kafka_brokers = dict(localhost=[dict(url="some_server", port=9092)])

overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", brokers=kafka_brokers)
async def on_preprocessed_signals_overriden(msg: TestMsg):
    print(f"Overriden:  {msg=}")


@overriden_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    print(f"Original:  {msg=}")


tester = Tester(overriden_app).using_local_kafka()


@tester.produces(brokers=kafka_brokers)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    print(overriden_app._override_brokers)
    print(tester._override_brokers)
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals_overriden.assert_called(
        timeout=5
    )
    await overriden_app.awaited_mocks.on_preprocessed_signals.assert_not_called(
        timeout=5
    )

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: zookeeper startup falied, generating a new port and retrying...
[INFO] fastkafka._testing.apache_kafka_broker: port=41979
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: kafka startup falied, generating a new port and retrying...
[INFO] fa

In [ ]:
kafka_brokers = dict(localhost=[dict(url="some_server", port=9092)])

overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", brokers=kafka_brokers)
async def on_preprocessed_signals_overriden(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@overriden_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


tester = Tester(overriden_app)


@tester.produces(topic="preprocessed_signals", brokers=kafka_brokers)
async def to_preprocessed_signals_test_override(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


@tester.consumes(auto_offset_reset="earliest")
async def on_predictions(msg: TestMsg):
    print(f"tester: {msg=}")
    pass


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals_test_override(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals_overriden.assert_called(
        timeout=5
    )
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['some_server:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_

In [ ]:
kafka_brokers = dict(localhost=[dict(url="some_server", port=9092)])

overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", brokers=kafka_brokers)
async def on_preprocessed_signals_overriden(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@overriden_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


tester = Tester(overriden_app).using_local_kafka()


@tester.produces(topic="preprocessed_signals", brokers=kafka_brokers)
async def to_preprocessed_signals_test_override(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


@tester.consumes(auto_offset_reset="earliest")
async def on_predictions(msg: TestMsg):
    print(f"tester: {msg=}")
    pass


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals_test_override(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals_overriden.assert_called(
        timeout=5
    )
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: zookeeper startup falied, generating a new port and retrying...
[INFO] fastkafka._testing.apache_kafka_broker: port=56977
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
stdout=, stderr=, returncode=1
[INFO] fastkafka._testing.apache_kafka_broker: kafka startup falied, generating a new port and retrying...
[INFO] fa

## Mirroring

In [ ]:
# | export


def mirror_producer(topic: str, producer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(producer_f).return_annotation

    msg_type_unwrapped = unwrap_list_type(unwrap_from_kafka_event(msg_type))

    async def skeleton_func(msg: BaseModel) -> None:
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "on_" + topic.replace(".", "_")

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type_unwrapped,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig  # type: ignore

    return mirror_func

In [ ]:
@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, _, _) in app._producers_store.items():
    mirror = mirror_producer(topic, producer_f)
    assert mirror.__name__ == "on_" + topic
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [ ]:
@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> KafkaEvent[List[TestMsg]]:
    pass


for topic, (producer_f, _, _, _) in app._producers_store.items():
    mirror = mirror_producer(topic, producer_f)
    assert mirror.__name__ == "on_" + topic.replace(".", "_")
    assert inspect.signature(mirror).parameters["msg"].annotation.__name__ == "TestMsg"

In [ ]:
# | export


def mirror_consumer(topic: str, consumer_f: Callable[..., Any]) -> Callable[..., Any]:
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    msg_type_unwrapped = unwrap_list_type(msg_type)

    async def skeleton_func(msg: BaseModel) -> BaseModel:
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name
    mirror_func.__name__ = "to_" + topic.replace(".", "_")

    # adjust arg and return val
    sig = sig.replace(
        parameters=[msg_type], return_annotation=msg_type_unwrapped.annotation
    )

    mirror_func.__signature__ = sig  # type: ignore
    return mirror_func

In [ ]:
for topic, (consumer_f, _, _, _, _) in app._consumers_store.items():
    mirror = mirror_consumer(topic, consumer_f)
    assert mirror.__name__ == "to_" + topic
    assert (
        inspect.signature(mirror).return_annotation.__name__ == TestMsg.__name__
    ), inspect.signature(mirror).return_annotation.__name__
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [ ]:
# | export


@patch
def create_mirrors(self: Tester) -> None:
    for app in self.apps:
        for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(topic, consumer_f)
            mirror_f = self.produces(  # type: ignore
                topic=topic,
                kafka_brokers=brokers,
            )(mirror_f)
            self.mirrors[consumer_f] = mirror_f
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, brokers, _) in app._producers_store.items():
            mirror_f = mirror_producer(topic, producer_f)
            mirror_f = self.consumes( # type: ignore
                topic=topic,
                kafka_brokers=brokers,
            )(mirror_f)
            setattr(self, mirror_f.__name__, mirror_f)

In [ ]:
kafka_brokers = 
    {"test_override":
    dict(localhost=[dict(url="some_server", port=9092)]),
    }

overriden_app = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092))
)


@overriden_app.consumes(topic="preprocessed_signals", kafka_brokers=kafka_brokers)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@overriden_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester(overriden_app).using_local_kafka() as tester:
    await tester.to_preprocessed_signals_test_override(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

In [ ]:
kafka_brokers = OverrideBrokers(
    "test_override",
    dict(
        localhost=[
            dict(url="some_server", port=9092)
        ]
    ),
)

overriden_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))

@overriden_app.consumes(topic="preprocessed_signals", kafka_brokers=kafka_brokers)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))

@overriden_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

async with Tester(overriden_app) as tester:
    await tester.to_preprocessed_signals_test_override(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await overriden_app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

In [ ]:
# Test mirroring with "." in topic names


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes(topic="this.should_work.now")
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces(topic="some.dots.my_topic")
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return [prediction]


async with Tester(second_app) as tester:
    await tester.to_this_should_work_now(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_some_dots_my_topic.assert_called(timeout=5)
print("ok")

In [ ]:
# Test batch mirroring


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg, meta: EventMetadata):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]


async with Tester(second_app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

In [ ]:
# Test KafkaEvent mirroring and consumer batching


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: List[TestMsg]):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> KafkaEvent[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return KafkaEvent(message=prediction, key=b"123")


async with Tester(second_app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

In [ ]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")

In [ ]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]).using_local_kafka() as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
print("ok")